In [43]:
import torch
import json
import cv2
from datetime import date
import numpy as np
import xml.etree.ElementTree as ET
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from PIL import Image
from ibug.face_detection import RetinaFacePredictor
from ibug.face_parsing import FaceParser as RTNetPredictor
from ibug.face_parsing.utils import label_colormap
%matplotlib inline

In [44]:
threshold = 0.8 # default = 0.8
weights = None # r"C:\mahmoud_dev\machine learning\segmentation\face_parsing\ibug\face_parsing\rtnet\weights\rtnet101-fcn-14.torch" # default = None
num_classes = 14 # default = 11
max_num_faces = 50 # default = 50

parser_encoder = 'rtnet50'
parser_decoder = 'fcn'
rotate_image = False
today = date.today()


if torch.cuda.is_available():
    device = 'cuda:0'
face_detector = RetinaFacePredictor(threshold=threshold, device=device, model=(RetinaFacePredictor.get_model('mobilenet0.25')))
face_parser = RTNetPredictor(device=device, ckpt=weights, encoder=parser_encoder, decoder=parser_decoder, num_classes=num_classes)

def get_image_pred(img, face_detector, face_parser):
    if rotate_image:
        img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)

    faces = face_detector(img, rgb=False)
    masks = face_parser.predict_img(img, faces, rgb=False)
    
    return faces, masks


Hybrid stages [True, True, True]


In [45]:
categories_list = ['background', 'skin', 'left_eyebrow', 'nose', 'upper_lip', 'inner_mouth', 'lower_lip',
                   'right_eyebrow', 'left_eye', 'hair', 'left_ear', 'right_ear', 'right_eye', 'glasses']

categories = [{"name":category, "id":i} for i, category in enumerate(categories_list)]

# Initialize COCO format dictionary
coco_dict = {
    'info': {"contributor":"mahmoud tabikh","date_created":date.today().strftime('%d-%m-%Y'),"description":"","url":"","version":"","year":""},
    'licenses': [{"name":"","id":0,"url":""}],
    'categories': categories,
    'images': [],
    'type': 'instances',
    'annotations': []
}

In [46]:
# Set paths and filenames
image_dir = r'D:\_Xchng\Mahmoud\segmenation\dataset\data\raw_images'
json_filepath = r'D:\_Xchng\Mahmoud\segmenation\dataset\data\instances_default.json'

In [47]:
def working_save_segmentation_coco(image_dir):
    # Loop through images in directory
    for image_id, filename in enumerate(os.listdir(image_dir)):
        if filename.endswith(tuple([".png", ".jpg"])):

            image_path = os.path.join(image_dir, filename)
            image = cv2.imread(image_path)

            image_dict = { 
                'id': image_id+1,
                'width': image.shape[1],
                'height': image.shape[0],
                'file_name': filename}
            coco_dict['images'].append(image_dict)

            faces, masks = get_image_pred(image, face_detector, face_parser)
            mask_arr, face = masks[0], faces[0] # assumes 1 face per image, loop for more faces.

            annotations_list = get_annotations_list(mask_arr, face, image_id+1, coco_dict["annotations"])
            for dict_ in annotations_list:
                coco_dict['annotations'].append(dict_)

    save_json(coco_dict)


def save_segmentation_yolo(image_dir):
    # Save segmentation results in YOLO format
    with open('segmentation_result.txt', 'w') as f:
        for i, obj in enumerate(objects):
            bbox = obj['bbox']
            segmentation_mask = segmentation_results[i]
            obj_center_x = (bbox[0] + bbox[2]) / 2.0
            obj_center_y = (bbox[1] + bbox[3]) / 2.0
            obj_width = bbox[2] - bbox[0]
            obj_height = bbox[3] - bbox[1]
            pixel_coords = np.argwhere(segmentation_mask == 1)
            pixel_coords_str = ','.join([f"{x[1]},{x[0]}" for x in pixel_coords])
            line = f"0 {obj_center_x} {obj_center_y} {obj_width} {obj_height} {pixel_coords_str}\n"
            f.write(line)


def get_annotations_list(mask_arr, face, image_id, dict_):
    # Create annotation dictionary for each unique mask value
    annotation_list = [] # i am making a list of lists instead of list of dicts
    for value in np.unique(mask_arr):
        if value != 0:
            annotation_id = len(dict_) + 1
            mask_bool = mask_arr == value
            annotation_dict = {
                'id': int(annotation_id),
                'image_id': int(image_id),
                'category_id': value,
                'segmentation': [],
                'area': np.sum(mask_bool),
                'bbox': face[:4].astype(int),
                'iscrowd': 0,
                'ignore': 0}
            # Create segmentation and bbox arrays
            ys, xs = np.where(mask_bool)
            segmentation = np.asarray(list(zip(xs, ys))).flatten().tolist()
            annotation_dict['segmentation'].append(segmentation)
            annotation_list.append(annotation_dict)
    # Add annotation to COCO dictionary
    return annotation_list

def save_json(coco_dict):
    class NpEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, np.integer):
                return int(obj)
            if isinstance(obj, np.floating):
                return float(obj)
            if isinstance(obj, np.ndarray):
                return obj.tolist()
            return super(NpEncoder, self).default(obj)
    Path(json_filepath).write_text(json.dumps(coco_dict, cls=NpEncoder, indent=3))
    print(f"json file written to {json_filepath}")

In [48]:
save_segmentation_coco(image_dir)

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\contours.cpp:197: error: (-210:Unsupported format or combination of formats) [Start]FindContours supports only CV_8UC1 images when mode != CV_RETR_FLOODFILL otherwise supports CV_32SC1 images only in function 'cvStartFindContours_Impl'
